In [2]:
import pandas as pd
import matplotlib 
import sklearn 
import sksurv
import numpy as np
import scipy.stats as stats
from scipy.stats import chisquare 

In [4]:
df = pd.read_csv("forecast_starting_data.csv")
df

,financial_active,net_payment_behaviour_tripd,promotion_flag,variable_rate_index,account_status_code,active_12_mths,bank_fico_buckets_20,charge_off_reason_code,mob,open_closed_flag,...,due_balance_2,due_balance_3,due_balance_4,due_balance_5,due_balance_6,due_balance_7,due_balance_8,snapshot,mth_code,industry
0,1.0,T,0,0,I 0-6,1.0,761+,NaN,42,1.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,202001,202001,B
1,0.0,P,0,0,A,1.0,761+,NaN,48,1.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,202001,202001,B
2,0.0,C,0,0,I 0-6,0.0,761+,NaN,244,1.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,202001,202001,A
3,1.0,R,0,0,I 0-6,1.0,621-640,NaN,76,1.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,202001,202001,A
4,1.0,P,0,0,I 0-6,1.0,761+,NaN,143,1.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,202001,202001,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19861,1.0,I,1,0,I 13-24,1.0,621-640,NaN,26,1.0,...,1363.04,0.0,0.0,0.0,0.0,0.0,0.0,202001,202001,A
19862,0.0,R,1,0,A,1.0,641-660,NaN,299,1.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,202001,202001,C
19863,1.0,I,0,0,I 0-6,1.0,741-760,NaN,4,1.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,202001,202001,C
19864,1.0,P,1,1,A,1.0,701-720,NaN,24,1.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,202001,202001,B


In [22]:
training_data = pd.read_csv("training_data.csv")

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (51) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [15]:
categorical = ["financial_active", "net_payment_behaviour_tripd", "promotion_flag", "account_status_code", "ever_delinquent_flg", "active_12_mths",
                "open_closed_flag", "purchase_active", "closed", "active", "charge_off", "charge_off_aged", "charge_off_bk", "writeoff_type_bko", 
                "writeoff_type_fraud_kiting", "writeoff_type_fraud_synthetic", "writeoff_type_deceased", "writeoff_type_other", "writeoff_type_aged",
                "writeoff_type_settlement", "writeoff_type_fraud_other", "writeoff_type_repo", "writeoff_type_null", "industry"]

categorical 

['financial_active',
 'net_payment_behaviour_tripd',
 'promotion_flag',
 'account_status_code',
 'ever_delinquent_flg',
 'active_12_mths',
 'open_closed_flag',
 'purchase_active',
 'closed',
 'active',
 'charge_off',
 'charge_off_aged',
 'charge_off_bk',
 'writeoff_type_bko',
 'writeoff_type_fraud_kiting',
 'writeoff_type_fraud_synthetic',
 'writeoff_type_deceased',
 'writeoff_type_other',
 'writeoff_type_aged',
 'writeoff_type_settlement',
 'writeoff_type_fraud_other',
 'writeoff_type_repo',
 'writeoff_type_null',
 'industry']

In [60]:
a = ['red', 'orange', 'yellow', 'green']
b = ['rabbit', 'dog', 'cat', 'dog']
c = ['xmas', 'thanksgiving', 'easter', 'halloween']
d = {'a':a, 'b':b, 'c':c}
test = pd.DataFrame(d)
test

,a,b,c
0,red,rabbit,xmas
1,orange,dog,thanksgiving
2,yellow,cat,easter
3,green,dog,halloween


In [61]:
test = pd.crosstab(test['a'],
                    [test['b'], test['c']], 
                    margins=True, margins_name="Total")

test

b         cat       dog              rabbit Total
c      easter halloween thanksgiving   xmas      
a                                                
green       0         1            0      0     1
orange      0         0            1      0     1
red         0         0            0      1     1
yellow      1         0            0      0     1
Total       1         1            1      1     4

In [46]:
categorical

['financial_active',
 'net_payment_behaviour_tripd',
 'promotion_flag',
 'account_status_code',
 'ever_delinquent_flg',
 'active_12_mths',
 'open_closed_flag',
 'purchase_active',
 'closed',
 'active',
 'charge_off',
 'charge_off_aged',
 'charge_off_bk',
 'writeoff_type_bko',
 'writeoff_type_fraud_kiting',
 'writeoff_type_fraud_synthetic',
 'writeoff_type_deceased',
 'writeoff_type_other',
 'writeoff_type_aged',
 'writeoff_type_settlement',
 'writeoff_type_fraud_other',
 'writeoff_type_repo',
 'writeoff_type_null',
 'industry']

In [48]:
col_array = [training_data['financial_active'], training_data['net_payment_behaviour_tripd'], 
            training_data['promotion_flag'], training_data['account_status_code'], 
            training_data['ever_delinquent_flg'], training_data['active_12_mths'], training_data['open_closed_flag'],
            training_data['purchase_active'], training_data['closed'], training_data['active'], training_data['charge_off'],
            training_data['charge_off_aged'], training_data['charge_off_bk'], training_data['writeoff_type_bko'],
            training_data['writeoff_type_fraud_kiting'], training_data['writeoff_type_fraud_synthetic'], 
            training_data['writeoff_type_deceased'], training_data['writeoff_type_other'], training_data['writeoff_type_aged'],
            training_data['writeoff_type_settlement'], training_data['writeoff_type_fraud_other'], training_data['writeoff_type_repo'],
            training_data['writeoff_type_null'], training_data['industry']]
len(col_array)

24

In [49]:
data = pd.crosstab(training_data['charge_off'],
                    col_array, margins=True, margins_name="Total")

In [66]:
data_crosstab = data
data_crosstab

financial_active              0.0                                      ...  \
net_payment_behaviour_tripd     C                                      ...   
promotion_flag                  0                                      ...   
account_status_code             A                                      ...   
ever_delinquent_flg           0.0                                      ...   
active_12_mths                0.0                             1.0      ...   
open_closed_flag              0.0    1.0                      0.0      ...   
purchase_active               0.0    0.0          1.0         0.0      ...   
closed                          0      0            0           0      ...   
active                          1      1            1           1      ...   
charge_off                      0      0            0           0      ...   
charge_off_aged                 0      0            0           0      ...   
charge_off_bk                   0      0            0           0      ...   
writeoff_type_bko               0      0            0           0      ...   
writeoff_type_fraud_kiting      0      0            0           0      ...   
writeoff_type_fraud_synthetic   0      0            0           0      ...   
writeoff_type_deceased          0      0            0           0      ...   
writeoff_type_other             0      0            0           0      ...   
writeoff_type_aged              0      0            0           0      ...   
writeoff_type_settlement        0      0            0           0      ...   
writeoff_type_fraud_other       0      0            0           0      ...   
writeoff_type_repo              0      0            0           0      ...   
writeoff_type_null              1      1            1           1      ...   
industry                        B  C   A    B   C   A   B   C   A   B  ...   
charge_off                                                             ...   
0                               3  1  73  182  49  11  39  11   3  13  ...   
1                               0  0   0    0   0   0   0   0   0   0  ...   
Total                           3  1  73  182  49  11  39  11   3  13  ...   

financial_active              1.0                                 Total  
net_payment_behaviour_tripd     T                                        
promotion_flag                  1                                        
account_status_code             W                                        
ever_delinquent_flg           0.0    1.0                                 
active_12_mths                1.0    0.0     1.0                         
open_closed_flag              1.0    1.0     1.0                         
purchase_active               1.0    0.0 1.0 0.0        1.0              
closed                          0      0   0   0          0              
active                          1      1   1   1          1              
charge_off                      0      0   0   0          0              
charge_off_aged                 0      0   0   0          0              
charge_off_bk                   0      0   0   0          0              
writeoff_type_bko               0      0   0   0          0              
writeoff_type_fraud_kiting      0      0   0   0          0              
writeoff_type_fraud_synthetic   0      0   0   0          0              
writeoff_type_deceased          0      0   0   0          0              
writeoff_type_other             0      0   0   0          0              
writeoff_type_aged              0      0   0   0          0              
writeoff_type_settlement        0      0   0   0          0              
writeoff_type_fraud_other       0      0   0   0          0              
writeoff_type_repo              0      0   0   0          0              
writeoff_type_null              1      1   1   1          1              
industry                        B  C   B   C   A   B  C   A  B           
charge_off                             